# Correlation Analysis: Skills, Experience & Salary

## Research Questions
1. Which skills correlate most with higher salaries?
2. Do certain skill combinations predict higher pay?
3. Is experience level the strongest salary predictor?

## Methods
- Pearson/Spearman correlation
- Partial correlation (controlling for experience)
- Correlation heatmaps

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import pearsonr, spearmanr
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('✓ Imports complete')

✓ Imports complete


## 1. Load and Prepare Data

In [3]:
# Load data with skills (use the 2025 filtered version with skills and work type)
df = pd.read_csv('data/processed/jobs_with_work_type_v2.csv', low_memory=False)

print(f"Total jobs: {len(df):,}")
print(f"\nColumns: {df.columns.tolist()}")

Total jobs: 46,610

Columns: ['job_id', 'title', 'company_name', 'location', 'state', 'posted_date_clean', 'year', 'month', 'quarter', 'salary_min', 'salary_max', 'salary_avg', 'has_salary', 'experience_level', 'work_type', 'is_remote', 'skills_text', 'skills_count', 'software_text', 'software_count', 'description', 'source', 'url', 'skills_extracted_text', 'skills_extracted_count']


In [4]:
# Filter to jobs with salary data
df_salary = df[df['salary_avg'].notna()].copy()

print(f"Jobs with salary data: {len(df_salary):,} ({len(df_salary)/len(df)*100:.1f}%)")
print(f"\nSalary range: ${df_salary['salary_avg'].min():,.0f} - ${df_salary['salary_avg'].max():,.0f}")
print(f"Mean salary: ${df_salary['salary_avg'].mean():,.0f}")
print(f"Median salary: ${df_salary['salary_avg'].median():,.0f}")

Jobs with salary data: 11,706 (25.1%)

Salary range: $9 - $550,000
Mean salary: $74,851
Median salary: $80,000


In [10]:
# Check suspected hourly wages
hourly_suspected = df_salary[df_salary['salary_avg'] < 100]

print(f"Jobs with salary < $100: {len(hourly_suspected):,}")
print(f"\nSample values:")
print(hourly_suspected['salary_avg'].value_counts().head(20))

# Check if they cluster around typical hourly rates ($15-$50/hr)
print(f"\nDistribution of suspected hourly wages:")
print(hourly_suspected['salary_avg'].describe())

Jobs with salary < $100: 3,624

Sample values:
salary_avg
57.5    209
30.0    165
33.5    135
32.5    108
15.0    102
20.0     99
25.0     92
60.0     91
35.0     88
22.5     86
55.0     85
17.5     80
50.0     80
12.5     78
40.0     76
27.5     72
37.5     71
45.0     70
30.5     69
65.0     68
Name: count, dtype: int64

Distribution of suspected hourly wages:
count    3624.000000
mean       40.224161
std        19.353863
min         9.000000
25%        25.000000
50%        35.000000
75%        55.000000
max        99.000000
Name: salary_avg, dtype: float64


In [11]:
# Convert hourly to annual (40 hrs/week × 52 weeks = 2080 hrs/year)
HOURS_PER_YEAR = 2080
HOURLY_THRESHOLD = 100  # Anything under $100 is likely hourly

df_salary['salary_converted'] = df_salary['salary_avg'].apply(
    lambda x: x * HOURS_PER_YEAR if x < HOURLY_THRESHOLD else x
)

# Show conversion impact
converted_count = (df_salary['salary_avg'] < HOURLY_THRESHOLD).sum()
print(f"✓ Converted {converted_count:,} hourly wages to annual")

print(f"\nBefore conversion:")
print(f"  Min: ${df_salary['salary_avg'].min():,.0f}")
print(f"  Mean: ${df_salary['salary_avg'].mean():,.0f}")

print(f"\nAfter conversion:")
print(f"  Min: ${df_salary['salary_converted'].min():,.0f}")
print(f"  Mean: ${df_salary['salary_converted'].mean():,.0f}")

# Use converted salary going forward
df_salary['salary_avg'] = df_salary['salary_converted']
df_salary = df_salary.drop(columns=['salary_converted'])

✓ Converted 3,624 hourly wages to annual

Before conversion:
  Min: $9
  Mean: $74,851

After conversion:
  Min: $100
  Mean: $100,741


In [12]:
# Analyze salary distribution for outliers
import numpy as np

print("\nSALARY DISTRIBUTION ANALYSIS")
print("="*60)

# Percentiles
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    val = df_salary['salary_avg'].quantile(p/100)
    print(f"{p:2}th percentile: ${val:>10,.0f}")

print("\n" + "="*60)

# Check extreme values
low_outliers = df_salary[df_salary['salary_avg'] < 30000]
print(f"\nJobs with salary < $30k: {len(low_outliers):,} ({len(low_outliers)/len(df_salary)*100:.1f}%)")

high_outliers = df_salary[df_salary['salary_avg'] > 200000]
print(f"Jobs with salary > $200k: {len(high_outliers):,} ({len(high_outliers)/len(df_salary)*100:.1f}%)")


SALARY DISTRIBUTION ANALYSIS
 1th percentile: $    20,800
 5th percentile: $    39,666
10th percentile: $    47,840
25th percentile: $    68,640
50th percentile: $    94,496
75th percentile: $   124,800
90th percentile: $   160,000
95th percentile: $   189,541
99th percentile: $   242,229


Jobs with salary < $30k: 267 (2.3%)
Jobs with salary > $200k: 447 (3.8%)


In [13]:
# Clean salary outliers
SALARY_MIN = 30000
SALARY_MAX = 200000

df_salary = df_salary[
    (df_salary['salary_avg'] >= SALARY_MIN) & 
    (df_salary['salary_avg'] <= SALARY_MAX)
].copy()

print(f"\n✓ Cleaned to salary range: ${SALARY_MIN:,} - ${SALARY_MAX:,}")
print(f"✓ Remaining jobs: {len(df_salary):,}")
print(f"✓ New mean: ${df_salary['salary_avg'].mean():,.0f}")
print(f"✓ New median: ${df_salary['salary_avg'].median():,.0f}")


✓ Cleaned to salary range: $30,000 - $200,000
✓ Remaining jobs: 10,992
✓ New mean: $97,381
✓ New median: $93,600


## 2. Create Skill Binary Columns

In [14]:
# Parse skills from skills_extracted_text column
def create_skill_columns(df):
    """Create binary columns for each skill"""
    
    # Get all unique skills
    all_skills = set()
    for skills_text in df['skills_extracted_text'].dropna():
        if skills_text:
            skills = [s.strip() for s in skills_text.split(',')]
            all_skills.update(skills)
    
    print(f"Found {len(all_skills)} unique skills")
    
    # Create binary columns
    for skill in sorted(all_skills):
        df[f'skill_{skill}'] = df['skills_extracted_text'].str.contains(
            skill, case=False, na=False
        ).astype(int)
    
    return df, sorted(all_skills)

df_salary, skills_list = create_skill_columns(df_salary)

print(f"\nCreated {len(skills_list)} skill columns")
print(f"Sample skills: {skills_list[:10]}")

Found 43 unique skills

Created 43 skill columns
Sample skills: ['a/b testing', 'agile', 'airflow', 'api', 'aws', 'azure', 'bigquery', 'business intelligence', 'data modeling', 'data visualization']


In [15]:
# Check skill prevalence
skill_cols = [f'skill_{s}' for s in skills_list]
skill_counts = df_salary[skill_cols].sum().sort_values(ascending=False)

print("Top 20 Most Common Skills:")
for col, count in skill_counts.head(20).items():
    skill_name = col.replace('skill_', '')
    pct = (count / len(df_salary)) * 100
    print(f"{skill_name:30} | {count:5,} jobs ({pct:5.1f}%)")

Top 20 Most Common Skills:
r                              | 3,268 jobs ( 29.7%)
sql                            | 2,621 jobs ( 23.8%)
business intelligence          | 1,971 jobs ( 17.9%)
excel                          | 1,760 jobs ( 16.0%)
python                         | 1,554 jobs ( 14.1%)
statistics                     | 1,436 jobs ( 13.1%)
tableau                        | 1,430 jobs ( 13.0%)
power bi                       | 1,306 jobs ( 11.9%)
data visualization             | 1,013 jobs (  9.2%)
machine learning               |   721 jobs (  6.6%)
etl                            |   625 jobs (  5.7%)
data warehouse                 |   538 jobs (  4.9%)
agile                          |   495 jobs (  4.5%)
data modeling                  |   491 jobs (  4.5%)
aws                            |   410 jobs (  3.7%)
oracle                         |   371 jobs (  3.4%)
sas                            |   362 jobs (  3.3%)
snowflake                      |   346 jobs (  3.1%)
azure              

## 3. Encode Experience Level

In [16]:
# Create numeric experience level
experience_mapping = {
    'Entry Level': 1,
    'Mid Level': 2,
    'Senior Level': 3,
    'Lead': 4,
    'Executive': 5,
    'Not Specified': np.nan
}

df_salary['experience_numeric'] = df_salary['experience_level'].map(experience_mapping)

# Filter to jobs with experience level
df_analysis = df_salary[df_salary['experience_numeric'].notna()].copy()

print(f"Jobs with both salary and experience: {len(df_analysis):,}")
print(f"\nExperience distribution:")
print(df_analysis['experience_level'].value_counts())

Jobs with both salary and experience: 0

Experience distribution:
Series([], Name: count, dtype: int64)


## 4. Skills vs Salary Correlation

In [ ]:
# Calculate correlation for each skill
skill_correlations = []

for skill in skills_list:
    col = f'skill_{skill}'
    
    # Only calculate if skill appears in at least 20 jobs
    if df_analysis[col].sum() < 20:
        continue
    
    # Pearson correlation
    corr, p_value = pearsonr(df_analysis[col], df_analysis['salary_avg'])
    
    # Mean salary comparison
    with_skill = df_analysis[df_analysis[col] == 1]['salary_avg'].mean()
    without_skill = df_analysis[df_analysis[col] == 0]['salary_avg'].mean()
    salary_diff = with_skill - without_skill
    
    skill_correlations.append({
        'skill': skill,
        'correlation': corr,
        'p_value': p_value,
        'n_jobs': df_analysis[col].sum(),
        'mean_salary_with': with_skill,
        'mean_salary_without': without_skill,
        'salary_premium': salary_diff,
        'premium_pct': (salary_diff / without_skill * 100) if without_skill > 0 else 0
    })

corr_df = pd.DataFrame(skill_correlations).sort_values('correlation', ascending=False)

print(f"\nAnalyzed {len(corr_df)} skills with 20+ occurrences")

In [ ]:
# Top skills positively correlated with salary
print("\n" + "="*80)
print("TOP 15 SKILLS CORRELATED WITH HIGHER SALARY")
print("="*80)

for idx, row in corr_df.head(15).iterrows():
    sig = "***" if row['p_value'] < 0.001 else "**" if row['p_value'] < 0.01 else "*" if row['p_value'] < 0.05 else ""
    print(f"{row['skill']:25} | r={row['correlation']:+.3f}{sig:3} | "
          f"Premium: ${row['salary_premium']:>7,.0f} ({row['premium_pct']:>+5.1f}%) | "
          f"{row['n_jobs']:>5,.0f} jobs")

In [ ]:
# Bottom skills (negative correlation)
print("\n" + "="*80)
print("SKILLS CORRELATED WITH LOWER SALARY")
print("="*80)

for idx, row in corr_df.tail(10).iterrows():
    sig = "***" if row['p_value'] < 0.001 else "**" if row['p_value'] < 0.01 else "*" if row['p_value'] < 0.05 else ""
    print(f"{row['skill']:25} | r={row['correlation']:+.3f}{sig:3} | "
          f"Penalty: ${row['salary_premium']:>7,.0f} ({row['premium_pct']:>+5.1f}%) | "
          f"{row['n_jobs']:>5,.0f} jobs")

In [ ]:
# Visualize top correlations
top_n = 20
plot_df = pd.concat([
    corr_df.head(top_n//2),
    corr_df.tail(top_n//2)
]).sort_values('correlation')

fig = go.Figure(go.Bar(
    x=plot_df['correlation'],
    y=plot_df['skill'],
    orientation='h',
    marker=dict(
        color=plot_df['correlation'],
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title="Correlation")
    ),
    text=plot_df['correlation'].apply(lambda x: f'{x:+.3f}'),
    textposition='outside'
))

fig.update_layout(
    title='Skills Most/Least Correlated with Salary',
    xaxis_title='Correlation Coefficient',
    yaxis_title='',
    height=600,
    template='plotly_dark'
)

fig.show()

## 5. Experience Level vs Salary

In [ ]:
# Experience level correlation
exp_corr, exp_p = pearsonr(df_analysis['experience_numeric'], df_analysis['salary_avg'])

print(f"\nExperience Level Correlation with Salary:")
print(f"Pearson r = {exp_corr:.3f} (p < 0.001)")
print(f"\nThis is {'STRONGER' if abs(exp_corr) > corr_df['correlation'].abs().max() else 'WEAKER'} than any individual skill")

In [ ]:
# Salary by experience level
exp_salary = df_analysis.groupby('experience_level')['salary_avg'].agg(['mean', 'median', 'count']).round(0)
exp_salary = exp_salary.reindex(['Entry Level', 'Mid Level', 'Senior Level', 'Lead', 'Executive'])

print("\nSalary by Experience Level:")
print("="*60)
print(exp_salary)

In [ ]:
# Visualize experience vs salary
fig = px.box(
    df_analysis,
    x='experience_level',
    y='salary_avg',
    category_orders={'experience_level': ['Entry Level', 'Mid Level', 'Senior Level', 'Lead', 'Executive']},
    title='Salary Distribution by Experience Level',
    labels={'salary_avg': 'Average Salary ($)', 'experience_level': 'Experience Level'},
    template='plotly_dark'
)

fig.update_traces(marker=dict(color='lightblue'))
fig.show()

## 6. Partial Correlation (Controlling for Experience)

In [ ]:
# Calculate partial correlation for top skills
def partial_correlation(x, y, z):
    """Calculate partial correlation between x and y, controlling for z"""
    r_xy = np.corrcoef(x, y)[0, 1]
    r_xz = np.corrcoef(x, z)[0, 1]
    r_yz = np.corrcoef(y, z)[0, 1]
    
    numerator = r_xy - (r_xz * r_yz)
    denominator = np.sqrt((1 - r_xz**2) * (1 - r_yz**2))
    
    return numerator / denominator if denominator != 0 else 0

# Calculate for top 20 skills
partial_corrs = []

for idx, row in corr_df.head(20).iterrows():
    skill_col = f'skill_{row["skill"]}'
    
    # Regular correlation
    regular_corr = row['correlation']
    
    # Partial correlation (controlling for experience)
    partial_corr = partial_correlation(
        df_analysis[skill_col].values,
        df_analysis['salary_avg'].values,
        df_analysis['experience_numeric'].values
    )
    
    partial_corrs.append({
        'skill': row['skill'],
        'regular_correlation': regular_corr,
        'partial_correlation': partial_corr,
        'difference': regular_corr - partial_corr
    })

partial_df = pd.DataFrame(partial_corrs)

In [ ]:
print("\n" + "="*80)
print("PARTIAL CORRELATION (Controlling for Experience Level)")
print("="*80)
print("\nSkills with INDEPENDENT effect on salary (after controlling for experience):\n")

for idx, row in partial_df.sort_values('partial_correlation', ascending=False).head(15).iterrows():
    print(f"{row['skill']:25} | "
          f"Regular: {row['regular_correlation']:+.3f} | "
          f"Partial: {row['partial_correlation']:+.3f} | "
          f"Diff: {row['difference']:+.3f}")

## 7. Skill Combinations

In [ ]:
# Analyze top skill combinations
from itertools import combinations

# Get top 10 skills by correlation
top_skills = corr_df.head(10)['skill'].tolist()

# Analyze 2-skill combinations
combo_results = []

for skill1, skill2 in combinations(top_skills, 2):
    col1 = f'skill_{skill1}'
    col2 = f'skill_{skill2}'
    
    # Jobs with both skills
    both = df_analysis[(df_analysis[col1] == 1) & (df_analysis[col2] == 1)]
    
    if len(both) >= 10:  # At least 10 jobs
        # Jobs with only skill1
        only1 = df_analysis[(df_analysis[col1] == 1) & (df_analysis[col2] == 0)]
        # Jobs with only skill2
        only2 = df_analysis[(df_analysis[col1] == 0) & (df_analysis[col2] == 1)]
        # Jobs with neither
        neither = df_analysis[(df_analysis[col1] == 0) & (df_analysis[col2] == 0)]
        
        combo_results.append({
            'skill1': skill1,
            'skill2': skill2,
            'n_both': len(both),
            'salary_both': both['salary_avg'].mean(),
            'salary_only1': only1['salary_avg'].mean() if len(only1) > 0 else 0,
            'salary_only2': only2['salary_avg'].mean() if len(only2) > 0 else 0,
            'salary_neither': neither['salary_avg'].mean() if len(neither) > 0 else 0
        })

combo_df = pd.DataFrame(combo_results)
combo_df['combo_premium'] = combo_df['salary_both'] - combo_df['salary_neither']
combo_df = combo_df.sort_values('salary_both', ascending=False)

In [ ]:
print("\n" + "="*80)
print("TOP 15 SKILL COMBINATIONS BY AVERAGE SALARY")
print("="*80)

for idx, row in combo_df.head(15).iterrows():
    print(f"{row['skill1']:20} + {row['skill2']:20} | "
          f"${row['salary_both']:>8,.0f} | "
          f"+${row['combo_premium']:>7,.0f} premium | "
          f"{row['n_both']:>4,.0f} jobs")

## 8. Correlation Heatmap

In [ ]:
# Create correlation matrix for top skills + salary + experience
top_20_skills = corr_df.head(20)['skill'].tolist()
skill_cols_subset = [f'skill_{s}' for s in top_20_skills]

# Select columns for correlation
corr_cols = skill_cols_subset + ['experience_numeric', 'salary_avg']
corr_matrix = df_analysis[corr_cols].corr()

# Rename for readability
rename_dict = {f'skill_{s}': s for s in top_20_skills}
rename_dict['experience_numeric'] = 'Experience Level'
rename_dict['salary_avg'] = 'Salary'
corr_matrix = corr_matrix.rename(columns=rename_dict, index=rename_dict)

In [ ]:
# Plot heatmap
plt.figure(figsize=(16, 14))
sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    vmin=-0.5,
    vmax=0.5,
    square=True,
    linewidths=0.5,
    cbar_kws={'label': 'Correlation Coefficient'}
)
plt.title('Correlation Heatmap: Top Skills, Experience & Salary', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('../visualizations/correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Heatmap saved to visualizations/correlation_heatmap.png")

## 9. Key Findings Summary

In [ ]:
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

# 1. Strongest predictor
top_skill_corr = corr_df.iloc[0]['correlation']
print(f"\n1. STRONGEST SALARY PREDICTOR:")
print(f"   Experience Level: r = {exp_corr:.3f}")
print(f"   Top Skill ({corr_df.iloc[0]['skill']}): r = {top_skill_corr:.3f}")
print(f"   → Experience is {'STRONGER' if abs(exp_corr) > abs(top_skill_corr) else 'WEAKER'}")

# 2. Top value skills
print(f"\n2. TOP 5 HIGH-VALUE SKILLS:")
for idx, row in corr_df.head(5).iterrows():
    print(f"   {row['skill']:20} → +${row['salary_premium']:>7,.0f} ({row['premium_pct']:>+5.1f}%)")

# 3. Skills independent of experience
print(f"\n3. SKILLS WITH INDEPENDENT EFFECT (after controlling for experience):")
for idx, row in partial_df.nlargest(5, 'partial_correlation').iterrows():
    print(f"   {row['skill']:20} → Partial r = {row['partial_correlation']:+.3f}")

# 4. Best skill combo
best_combo = combo_df.iloc[0]
print(f"\n4. HIGHEST-PAYING SKILL COMBINATION:")
print(f"   {best_combo['skill1']} + {best_combo['skill2']}")
print(f"   → ${best_combo['salary_both']:,.0f} average salary")
print(f"   → +${best_combo['combo_premium']:,.0f} premium over no skills")

print("\n" + "="*80)

## 10. Export Results

In [ ]:
# Save correlation results
corr_df.to_csv('../data/processed/skill_salary_correlations.csv', index=False)
combo_df.to_csv('../data/processed/skill_combinations_analysis.csv', index=False)
partial_df.to_csv('../data/processed/partial_correlations.csv', index=False)

print("\n✓ Results exported to data/processed/")
print("  - skill_salary_correlations.csv")
print("  - skill_combinations_analysis.csv")
print("  - partial_correlations.csv")